In [1]:
import healpy as hp
import numpy as np
import xarray as xr
import pandas as pd
import intake
import matplotlib.pyplot as plt

## Functions

In [2]:
# Load Catalog
cat = intake.open_catalog(
    "https://data.nextgems-h2020.eu/catalog.yaml"
)
cat_hera = intake.open_catalog("https://tcodata.mpimet.mpg.de/internal.yaml")
print(f"Experiments in Catalog: {list(cat.keys())}")

Experiments in Catalog: ['ICON', 'IFS', 'FESOM']


# Healpix

In [3]:
# Example data
zoom_use = 8

t1 = "2020-01-01"
t2 = "2025-01-01"

t1_AMIP = "1979-01-01"  #"1979-03-01" #"1979-01-02" #"1979-04-25" 
t2_AMIP = "1979-01-01"  #"1979-03-01" #"1979-01-02" #"1979-04-25" 

test_data_ngc4008 = (
    cat["ICON"]["ngc4008"](time="P1D", zoom=zoom_use, chunks="auto")
    .to_dask()
    .sel(time=slice(t1, t2))
)
test_data_ngcAMIP = (
    cat["ICON.C5.AMIP_CNTL"](time="P1D", zoom=zoom_use, chunks="auto")
    .to_dask()
    .sel(time=slice(t1_AMIP, t2_AMIP))
)

/sw/spack-levante/mambaforge-23.1.0-1-Linux-x86_64-3boc6i/lib/python3.10/site-packages/intake_xarray/base.py:21: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  'dims': dict(self._ds.dims),
/sw/spack-levante/mambaforge-23.1.0-1-Linux-x86_64-3boc6i/lib/python3.10/site-packages/intake_xarray/base.py:21: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  'dims': dict(self._ds.dims),


## Get Lon-Lat Coordinates

In [4]:
def attach_coords(ds, nside, nest_tf):
    lons, lats = hp.pix2ang(nside, ds.cell.values, nest=nest_tf, lonlat=True)
    return ds.assign_coords(
        lat=(("cell",), lats, {"units": "degrees_north"}),
        lon=(("cell",), lons, {"units": "degrees_east"}),
    )

In [5]:
ngc4008 = attach_coords(test_data_ngc4008, test_data_ngc4008.crs.healpix_nside, True)
ngcAMIP = attach_coords(test_data_ngcAMIP, test_data_ngc4008.crs.healpix_nside, True)

# Prepare COARE3.6 Algorithm

# Tropical Pacific

In [6]:
TOP = 20
BOTTOM = -20 
LEFT = 150
RIGHT = 250

def tropical_pacific(ds):
    return (ds.lat > BOTTOM) & (ds.lat < TOP) & (ds.lon > LEFT) & (ds.lon < RIGHT)

def ocean(ds):
    return ds.ocean_fraction_surface == 1

In [7]:
ngc4008_ocean = (ngc4008
                    .where(tropical_pacific(ocean(ngc4008)).compute(), drop=True)
                    .sel(time=slice('2020-01-02T00:00:00.000000000', '2020-01-31T00:00:00.000000000'))
                   )
ngc4008_ocean

<xarray.Dataset> Size: 15GB
Dimensions:                              (time: 30, depth_half: 73,
                                          cell: 74824, level_full: 90,
                                          soil_depth_water_level: 5,
                                          depth_full: 72, level_half: 91,
                                          soil_depth_energy_level: 5, crs: 1)
Coordinates:
  * crs                                  (crs) float32 4B nan
  * depth_full                           (depth_full) float32 288B 1.0 ... 5....
  * depth_half                           (depth_half) float32 292B 0.0 ... 5....
  * level_full                           (level_full) float64 720B 1.0 ... 90.0
  * level_half                           (level_half) float64 728B 1.0 ... 91.0
  * soil_depth_energy_level              (soil_depth_energy_level) float32 20B ...
  * soil_depth_water_level               (soil_depth_water_level) float32 20B ...
  * time                                 (time) datetime64[ns] 240B 2020-01-0...
    lat                                  (cell) float64 599kB 13.09 ... -0.1492
    lon                                  (cell) float64 599kB 150.1 ... 225.0
Dimensions without coordinates: cell
Data variables: (12/103)
    A_tracer_v_to                        (time, depth_half, cell) float32 655MB dask.array<chunksize=(7, 11, 529), meta=np.ndarray>
    FrshFlux_IceSalt                     (time, cell) float32 9MB dask.array<chunksize=(30, 529), meta=np.ndarray>
    FrshFlux_TotalIce                    (time, cell) float32 9MB dask.array<chunksize=(30, 529), meta=np.ndarray>
    Qbot                                 (time, cell) float32 9MB dask.array<chunksize=(30, 529), meta=np.ndarray>
    Qtop                                 (time, cell) float32 9MB dask.array<chunksize=(30, 529), meta=np.ndarray>
    Wind_Speed_10m                       (time, cell) float32 9MB dask.array<chunksize=(30, 529), meta=np.ndarray>
    ...                                   ...
    vas                                  (time, cell) float32 9MB dask.array<chunksize=(30, 529), meta=np.ndarray>
    w                                    (time, depth_half, cell) float32 655MB dask.array<chunksize=(7, 11, 529), meta=np.ndarray>
    wa_phy                               (time, level_half, cell) float32 817MB dask.array<chunksize=(7, 13, 529), meta=np.ndarray>
    zg                                   (level_full, cell) float32 27MB dask.array<chunksize=(60, 65649), meta=np.ndarray>
    zghalf                               (level_half, cell) float32 27MB dask.array<chunksize=(62, 65649), meta=np.ndarray>
    zos                                  (time, cell) float32 9MB dask.array<chunksize=(30, 529), meta=np.ndarray>

In [8]:
ocean_current_speed = ((ngc4008_ocean.u.sel(depth_full=1.0)**2 + ngc4008_ocean.v.sel(depth_full=1.0)**2) **(0.5)).compute() #m/s
ocean_current_speed

<xarray.DataArray (time: 30, cell: 74824)> Size: 9MB
array([[0.1722446 , 0.18916023, 0.22005166, ..., 0.5678311 , 0.67856747,
        0.6562794 ],
       [0.26359785, 0.25185907, 0.25578845, ..., 0.36861694, 0.52958703,
        0.46197656],
       [0.14967398, 0.14722508, 0.17184538, ..., 0.29149276, 0.41137174,
        0.35016957],
       ...,
       [0.26020586, 0.23881966, 0.25320134, ..., 0.7637268 , 0.7641452 ,
        0.7625649 ],
       [0.23064142, 0.21162033, 0.2186482 , ..., 0.7676794 , 0.7960406 ,
        0.7884041 ],
       [0.25225902, 0.24003886, 0.24147321, ..., 0.6979028 , 0.75216013,
        0.7501539 ]], dtype=float32)
Coordinates:
    depth_full  float32 4B 1.0
  * time        (time) datetime64[ns] 240B 2020-01-02 2020-01-03 ... 2020-01-31
    lat         (cell) float64 599kB 13.09 13.25 13.4 ... -0.2984 -0.1492
    lon         (cell) float64 599kB 150.1 150.3 150.1 ... 225.2 224.8 225.0
Dimensions without coordinates: cell

In [9]:
sfc_wind_10m_speed  = ngc4008_ocean.sfcwind.compute() #m/s
sfc_wind_10m_speed

<xarray.DataArray 'sfcwind' (time: 30, cell: 74824)> Size: 9MB
array([[ 9.034295 ,  8.8692045,  8.907172 , ...,  5.935851 ,  5.871426 ,
         5.7962103],
       [ 9.288168 ,  9.384019 ,  9.404886 , ...,  7.300605 ,  7.4166303,
         7.395815 ],
       [ 8.605442 ,  8.505821 ,  8.360179 , ...,  6.5694876,  6.8265443,
         6.7803354],
       ...,
       [10.20993  , 10.149313 , 10.107293 , ...,  8.852383 ,  8.926908 ,
         8.847944 ],
       [ 8.549884 ,  8.48176  ,  8.552181 , ...,  7.176012 ,  7.1610126,
         7.1421456],
       [ 9.342953 ,  9.284791 ,  9.205609 , ...,  5.77776  ,  5.6593037,
         5.727364 ]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 240B 2020-01-02 2020-01-03 ... 2020-01-31
    lat      (cell) float64 599kB 13.09 13.25 13.4 ... -0.2984 -0.2984 -0.1492
    lon      (cell) float64 599kB 150.1 150.3 150.1 150.5 ... 225.2 224.8 225.0
Dimensions without coordinates: cell
Attributes:
    cell_methods:  time: mean cell: mean
    component:     atmo
    grid_mapping:  crs
    long_name:     10m windspeed
    units:         m s-1
    vgrid:         height_10m

#### Water-Relative Wind Speed

In [10]:
water_relative_wind_speed = (sfc_wind_10m_speed - ocean_current_speed).rename('u') #m/s
water_relative_wind_speed

<xarray.DataArray 'u' (time: 30, cell: 74824)> Size: 9MB
array([[8.86205  , 8.680044 , 8.68712  , ..., 5.36802  , 5.1928587,
        5.1399307],
       [9.02457  , 9.13216  , 9.149097 , ..., 6.931988 , 6.887043 ,
        6.9338384],
       [8.455768 , 8.358596 , 8.1883335, ..., 6.2779946, 6.4151726,
        6.430166 ],
       ...,
       [9.949724 , 9.910493 , 9.854092 , ..., 8.088655 , 8.162763 ,
        8.08538  ],
       [8.3192425, 8.27014  , 8.333533 , ..., 6.408333 , 6.364972 ,
        6.3537416],
       [9.090693 , 9.044752 , 8.964136 , ..., 5.0798573, 4.9071436,
        4.97721  ]], dtype=float32)
Coordinates:
  * time        (time) datetime64[ns] 240B 2020-01-02 2020-01-03 ... 2020-01-31
    lat         (cell) float64 599kB 13.09 13.25 13.4 ... -0.2984 -0.1492
    lon         (cell) float64 599kB 150.1 150.3 150.1 ... 225.2 224.8 225.0
    depth_full  float32 4B 1.0
Dimensions without coordinates: cell

In [11]:
zu = xr.DataArray(10.0 * np.ones_like(water_relative_wind_speed), dims=water_relative_wind_speed.dims, name="zu") #m
zu

<xarray.DataArray 'zu' (time: 30, cell: 74824)> Size: 9MB
array([[10., 10., 10., ..., 10., 10., 10.],
       [10., 10., 10., ..., 10., 10., 10.],
       [10., 10., 10., ..., 10., 10., 10.],
       ...,
       [10., 10., 10., ..., 10., 10., 10.],
       [10., 10., 10., ..., 10., 10., 10.],
       [10., 10., 10., ..., 10., 10., 10.]], dtype=float32)
Dimensions without coordinates: time, cell

#### Surface Temperature

In [12]:
sfc_temp_2m_ocean   = (ngc4008_ocean.tas - 273.15).rename('t').compute() #deg C
sfc_temp_2m_ocean

<xarray.DataArray 't' (time: 30, cell: 74824)> Size: 9MB
array([[26.344055, 26.318542, 26.314362, ..., 24.56845 , 24.645813,
        24.600128],
       [25.621094, 25.54187 , 25.507477, ..., 24.030975, 24.236206,
        24.216919],
       [26.00592 , 25.966248, 25.911163, ..., 23.155304, 23.306213,
        23.310425],
       ...,
       [24.955475, 24.9357  , 24.921478, ..., 23.659454, 23.637268,
        23.611908],
       [24.625763, 24.599945, 24.606598, ..., 23.35907 , 23.400269,
        23.390656],
       [24.44757 , 24.411621, 24.382385, ..., 23.881683, 23.848938,
        23.862488]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 240B 2020-01-02 2020-01-03 ... 2020-01-31
    lat      (cell) float64 599kB 13.09 13.25 13.4 ... -0.2984 -0.2984 -0.1492
    lon      (cell) float64 599kB 150.1 150.3 150.1 150.5 ... 225.2 224.8 225.0
Dimensions without coordinates: cell

In [13]:
zt = xr.DataArray(2.0 * np.ones_like(sfc_temp_2m_ocean), dims=sfc_temp_2m_ocean.dims, name="zt") #m
zt

<xarray.DataArray 'zt' (time: 30, cell: 74824)> Size: 9MB
array([[2., 2., 2., ..., 2., 2., 2.],
       [2., 2., 2., ..., 2., 2., 2.],
       [2., 2., 2., ..., 2., 2., 2.],
       ...,
       [2., 2., 2., ..., 2., 2., 2.],
       [2., 2., 2., ..., 2., 2., 2.],
       [2., 2., 2., ..., 2., 2., 2.]], dtype=float32)
Dimensions without coordinates: time, cell

#### Sea Level Pressure

In [14]:
surface_pressure = (ngc4008_ocean.pres_sfc.rename('P')/100).compute() #hPa
surface_pressure

<xarray.DataArray 'P' (time: 30, cell: 74824)> Size: 9MB
array([[1015.2361 , 1015.315  , 1015.38245, ..., 1012.97656, 1012.9519 ,
        1012.97314],
       [1013.9559 , 1014.04614, 1014.15045, ..., 1013.2154 , 1013.15204,
        1013.1356 ],
       [1012.2109 , 1012.3007 , 1012.4143 , ..., 1013.4935 , 1013.43665,
        1013.44116],
       ...,
       [1012.40906, 1012.48944, 1012.6191 , ..., 1008.7795 , 1008.74866,
        1008.776  ],
       [1012.18823, 1012.2279 , 1012.32495, ..., 1009.4016 , 1009.39166,
        1009.38776],
       [1012.2004 , 1012.2679 , 1012.3591 , ..., 1009.5139 , 1009.5228 ,
        1009.50586]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 240B 2020-01-02 2020-01-03 ... 2020-01-31
    lat      (cell) float64 599kB 13.09 13.25 13.4 ... -0.2984 -0.2984 -0.1492
    lon      (cell) float64 599kB 150.1 150.3 150.1 150.5 ... 225.2 224.8 225.0
Dimensions without coordinates: cell

#### Specific Humidity

In [15]:
specific_humidity = ngc4008_ocean.hus.sel(level_full=90).compute()
specific_humidity

<xarray.DataArray 'hus' (time: 30, cell: 74824)> Size: 9MB
array([[0.01497743, 0.0150391 , 0.01498516, ..., 0.01546783, 0.01551762,
        0.0155586 ],
       [0.01520208, 0.01514037, 0.01500137, ..., 0.01491303, 0.01490379,
        0.01496258],
       [0.01501465, 0.01501716, 0.01487742, ..., 0.01487772, 0.01489483,
        0.01496589],
       ...,
       [0.01468331, 0.01457293, 0.01441218, ..., 0.01393633, 0.01399351,
        0.01384742],
       [0.01428537, 0.01426294, 0.01422205, ..., 0.01402737, 0.01401542,
        0.01403136],
       [0.01403756, 0.01399428, 0.01397589, ..., 0.01374697, 0.01380444,
        0.0137968 ]], dtype=float32)
Coordinates:
    level_full  float64 8B 90.0
  * time        (time) datetime64[ns] 240B 2020-01-02 2020-01-03 ... 2020-01-31
    lat         (cell) float64 599kB 13.09 13.25 13.4 ... -0.2984 -0.1492
    lon         (cell) float64 599kB 150.1 150.3 150.1 ... 225.2 224.8 225.0
Dimensions without coordinates: cell
Attributes:
    cell_methods:   time: mean cell: mean
    component:      atmo
    grid_mapping:   crs
    long_name:      Specific humidity
    standard_name:  specific_humidity
    vgrid:          reference

#### Relative Humidity

In [16]:
def compute_relative_humidity(specific_humidity, sfc_temp_2m_ocean, surface_pressure):
    # Compute actual vapor pressure (Pa)
    e = (specific_humidity * surface_pressure) / (0.622 + 0.378 * specific_humidity)
    
    # Compute saturation vapor pressure (Pa)
    temp_celsius = sfc_temp_2m_ocean
    e_s = 6.112 * np.exp(17.67 * temp_celsius / (temp_celsius + 243.5)) * 100  # Convert hPa to Pa
    
    # Compute relative humidity (%)
    relative_humidity = (e / e_s) * 100
    
    # Return as xarray DataArray
    relative_humidity = xr.DataArray(relative_humidity, coords=specific_humidity.coords, dims=specific_humidity.dims)
    
    return relative_humidity

In [17]:
relative_humidity = compute_relative_humidity(specific_humidity, sfc_temp_2m_ocean, surface_pressure).rename('rh').compute() #%
relative_humidity

<xarray.DataArray 'rh' (time: 30, cell: 74824)> Size: 9MB
array([[0.70616776, 0.7101743 , 0.7078719 , ..., 0.8084844 , 0.8072959 ,
        0.8116414 ],
       [0.7470839 , 0.74765706, 0.74244756, ..., 0.8054899 , 0.79508245,
        0.79910177],
       [0.72005886, 0.7219371 , 0.7176992 , ..., 0.84741443, 0.8406255 ,
        0.84438884],
       ...,
       [0.74985784, 0.7452085 , 0.7377806 , ..., 0.76684177, 0.77096826,
        0.76417524],
       [0.7440721 , 0.7440923 , 0.74175274, ..., 0.7864127 , 0.7837872 ,
        0.7851243 ],
       [0.73912865, 0.7385074 , 0.7389059 , ..., 0.74699324, 0.75157654,
        0.7505388 ]], dtype=float32)
Coordinates:
    level_full  float64 8B 90.0
  * time        (time) datetime64[ns] 240B 2020-01-02 2020-01-03 ... 2020-01-31
    lat         (cell) float64 599kB 13.09 13.25 13.4 ... -0.2984 -0.1492
    lon         (cell) float64 599kB 150.1 150.3 150.1 ... 225.2 224.8 225.0
Dimensions without coordinates: cell

In [18]:
zq = xr.DataArray(12.5 * np.ones_like(relative_humidity), dims=relative_humidity.dims, name="zq") #m
zq

<xarray.DataArray 'zq' (time: 30, cell: 74824)> Size: 9MB
array([[12.5, 12.5, 12.5, ..., 12.5, 12.5, 12.5],
       [12.5, 12.5, 12.5, ..., 12.5, 12.5, 12.5],
       [12.5, 12.5, 12.5, ..., 12.5, 12.5, 12.5],
       ...,
       [12.5, 12.5, 12.5, ..., 12.5, 12.5, 12.5],
       [12.5, 12.5, 12.5, ..., 12.5, 12.5, 12.5],
       [12.5, 12.5, 12.5, ..., 12.5, 12.5, 12.5]], dtype=float32)
Dimensions without coordinates: time, cell

#### Sea Water Temperature

In [19]:
sea_water_temperature = ngc4008_ocean.to.sel(depth_full=1.0).rename('ts').compute() #deg C
sea_water_temperature

<xarray.DataArray 'ts' (time: 30, cell: 74824)> Size: 9MB
array([[26.969748, 26.925465, 26.923418, ..., 25.246246, 25.452166,
        25.526434],
       [26.933075, 26.847542, 26.838388, ..., 25.246857, 25.455105,
        25.529215],
       [26.944397, 26.882042, 26.869621, ..., 24.97156 , 25.300156,
        25.399244],
       ...,
       [26.824255, 26.841019, 26.808329, ..., 23.921316, 23.80934 ,
        23.774044],
       [26.81828 , 26.83064 , 26.810097, ..., 23.965221, 23.9441  ,
        23.939121],
       [26.805344, 26.814785, 26.798412, ..., 24.050106, 24.04928 ,
        24.032396]], dtype=float32)
Coordinates:
    depth_full  float32 4B 1.0
  * time        (time) datetime64[ns] 240B 2020-01-02 2020-01-03 ... 2020-01-31
    lat         (cell) float64 599kB 13.09 13.25 13.4 ... -0.2984 -0.1492
    lon         (cell) float64 599kB 150.1 150.3 150.1 ... 225.2 224.8 225.0
Dimensions without coordinates: cell
Attributes:
    cell_methods:   time: mean cell: mean
    component:      ocean
    grid_mapping:   crs
    long_name:      sea water potential temperature
    standard_name:  sea_water_potential_temperature
    units:          degC
    vgrid:          depth_below_sea

#### Shortwave Down

In [20]:
shortwave_down = ngc4008_ocean.rsds.rename('sw_dn').compute() #W m-2
shortwave_down

<xarray.DataArray 'sw_dn' (time: 30, cell: 74824)> Size: 9MB
array([[240.33995, 237.9965 , 230.44434, ..., 283.05945, 269.04068,
        267.45425],
       [193.20917, 166.81006, 152.97847, ..., 275.29086, 281.92047,
        283.28543],
       [216.82233, 230.11942, 235.09982, ..., 262.15793, 251.22832,
        246.56552],
       ...,
       [247.32367, 253.82993, 254.10721, ..., 261.60327, 256.6684 ,
        277.5008 ],
       [194.29907, 191.43042, 202.52908, ..., 204.09554, 244.58377,
        242.01027],
       [229.83737, 239.67374, 241.6439 , ..., 284.6958 , 274.49756,
        280.00693]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 240B 2020-01-02 2020-01-03 ... 2020-01-31
    lat      (cell) float64 599kB 13.09 13.25 13.4 ... -0.2984 -0.2984 -0.1492
    lon      (cell) float64 599kB 150.1 150.3 150.1 150.5 ... 225.2 224.8 225.0
Dimensions without coordinates: cell
Attributes:
    cell_methods:   time: mean cell: mean
    component:      atmo
    grid_mapping:   crs
    long_name:      surface downwelling shortwave radiation
    standard_name:  surface_downwelling_shortwave_flux_in_air
    units:          W m-2
    vgrid:          surface

#### Longwave Down

In [21]:
longwave_down = ngc4008_ocean.rlds.rename('lw_dn').compute() #W m-2
longwave_down

<xarray.DataArray 'lw_dn' (time: 30, cell: 74824)> Size: 9MB
array([[401.7751 , 405.5091 , 407.49884, ..., 399.0682 , 401.4366 ,
        402.05536],
       [409.74506, 403.46497, 398.36105, ..., 403.75113, 398.00458,
        403.1691 ],
       [404.99414, 403.47427, 405.1921 , ..., 403.4659 , 400.0114 ,
        405.41528],
       ...,
       [399.5405 , 396.5104 , 395.8041 , ..., 404.08774, 402.802  ,
        402.13223],
       [408.34714, 406.1521 , 404.87982, ..., 394.7545 , 393.95242,
        391.79944],
       [397.21182, 397.05048, 397.11084, ..., 383.209  , 385.0673 ,
        385.11508]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 240B 2020-01-02 2020-01-03 ... 2020-01-31
    lat      (cell) float64 599kB 13.09 13.25 13.4 ... -0.2984 -0.2984 -0.1492
    lon      (cell) float64 599kB 150.1 150.3 150.1 150.5 ... 225.2 224.8 225.0
Dimensions without coordinates: cell
Attributes:
    cell_methods:   time: mean cell: mean
    component:      atmo
    grid_mapping:   crs
    long_name:      surface downwelling longwave radiation
    standard_name:  surface_downwelling_longwave_flux_in_air
    units:          W m-2
    vgrid:          surface

#### Lat / Lon

In [22]:
lon_2d = np.tile(water_relative_wind_speed.lon, (len(water_relative_wind_speed.time), 1)).T
longitude = xr.DataArray(lon_2d, 
                         dims=["cell", "time"], 
                         coords={"cell": np.arange(len(water_relative_wind_speed.lat)), "time": water_relative_wind_speed.time}, 
                         name="lon"
                        ).T #deg lon

In [23]:
lat_2d = np.tile(water_relative_wind_speed.lat, (len(water_relative_wind_speed.time), 1)).T
latitude = xr.DataArray(lat_2d, 
                        dims=["cell", "time"], 
                        coords={"cell": np.arange(len(water_relative_wind_speed.lat)), "time": water_relative_wind_speed.time}, 
                        name="lat"
                       ).T #deg lat

#### PBL height

In [24]:
zi = xr.DataArray(600.0 * np.ones_like(longitude), dims=longitude.dims, name="zi") #m
zi

<xarray.DataArray 'zi' (time: 30, cell: 74824)> Size: 18MB
array([[600., 600., 600., ..., 600., 600., 600.],
       [600., 600., 600., ..., 600., 600., 600.],
       [600., 600., 600., ..., 600., 600., 600.],
       ...,
       [600., 600., 600., ..., 600., 600., 600.],
       [600., 600., 600., ..., 600., 600., 600.],
       [600., 600., 600., ..., 600., 600., 600.]])
Dimensions without coordinates: time, cell

#### Rain Rate

In [25]:
# 1 kg m-2 s-1 = 3600 mm/hr
rain_rate = (ngc4008_ocean.pr * 3600).rename('rain').compute()
rain_rate.attrs['units'] = 'mm/hr'
rain_rate #mm/hr

<xarray.DataArray 'rain' (time: 30, cell: 74824)> Size: 9MB
array([[3.5223637e-02, 2.6687836e-02, 1.0296788e-02, ..., 6.5872753e-03,
        1.1831311e-02, 2.0898113e-02],
       [1.3090588e-02, 6.2136896e-02, 6.8532519e-02, ..., 5.0319727e-03,
        3.5137078e-03, 1.9368237e-03],
       [2.1050760e-04, 1.8587963e-04, 2.8001303e-03, ..., 9.6445000e-03,
        1.1146279e-02, 7.9409666e-03],
       ...,
       [1.0139843e-04, 4.3325515e-05, 1.1277922e-06, ..., 9.8744861e-04,
        8.9296338e-04, 7.1145361e-04],
       [1.9390061e-03, 1.1575641e-03, 5.5400177e-04, ..., 2.6701080e-02,
        2.0227520e-02, 6.4108092e-03],
       [6.2214388e-03, 2.0694903e-03, 1.3270733e-02, ..., 1.2875781e-08,
        1.0069394e-09, 1.8658231e-08]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 240B 2020-01-02 2020-01-03 ... 2020-01-31
    lat      (cell) float64 599kB 13.09 13.25 13.4 ... -0.2984 -0.2984 -0.1492
    lon      (cell) float64 599kB 150.1 150.3 150.1 150.5 ... 225.2 224.8 225.0
Dimensions without coordinates: cell
Attributes:
    units:    mm/hr

#### Sea Surface Salinity

In [26]:
sea_surface_salinity = ngc4008_ocean.so.sel(depth_full=1.0).rename('Ss').compute() #psu
sea_surface_salinity

<xarray.DataArray 'Ss' (time: 30, cell: 74824)> Size: 9MB
array([[34.158924, 34.160885, 34.15134 , ..., 34.683002, 34.667255,
        34.649857],
       [34.163498, 34.180305, 34.159336, ..., 34.68815 , 34.658997,
        34.64287 ],
       [34.169815, 34.193535, 34.17271 , ..., 34.724934, 34.67805 ,
        34.65711 ],
       ...,
       [34.210293, 34.210613, 34.250156, ..., 35.045757, 34.996006,
        35.00015 ],
       [34.210476, 34.21037 , 34.22867 , ..., 35.076225, 35.05618 ,
        35.065998],
       [34.228973, 34.21692 , 34.218513, ..., 35.091686, 35.083298,
        35.08554 ]], dtype=float32)
Coordinates:
    depth_full  float32 4B 1.0
  * time        (time) datetime64[ns] 240B 2020-01-02 2020-01-03 ... 2020-01-31
    lat         (cell) float64 599kB 13.09 13.25 13.4 ... -0.2984 -0.1492
    lon         (cell) float64 599kB 150.1 150.3 150.1 ... 225.2 224.8 225.0
Dimensions without coordinates: cell
Attributes:
    cell_methods:   time: mean cell: mean
    component:      ocean
    grid_mapping:   crs
    long_name:      sea water salinity
    standard_name:  sea_water_salinity
    units:          psu
    vgrid:          depth_below_sea

#### JD

In [27]:
data = ngc4008_ocean["time"]
data["time"] = pd.to_datetime(data["time"])

# Convert to Julian day where Jan 1 00:00 UTC = 0
def to_julian_day(time):
    time = pd.Timestamp(time)
    start_of_year = pd.Timestamp(year=time.year, month=1, day=1)
    return (time - start_of_year) / pd.Timedelta(days=1)

# Apply the conversion function to the time coordinate using xarray.apply_ufunc
julian_days = xr.apply_ufunc(
    to_julian_day,
    data["time"],
    vectorize=True,  # This allows the function to be applied element-wise
    dask='parallelized',  # This enables parallel computation if using dask
    output_dtypes=[float]  # The output type is float
)

# Assign the julian_days to a new coordinate "jd" in the data array
jd = data.assign_coords(jd=("time", julian_days.data)).rename('jd').jd
jd = jd.reset_coords(drop=True)

abc = xr.DataArray(3 * np.ones_like(longitude), dims=longitude.dims, name="zi")

# Create new time values array from 1 to 365
new_values = np.arange(1, len(jd.time)+1).reshape(-1, 1)

# Broadcast new values to match the shape of zi
new_data = np.tile(new_values, (1, abc.shape[1]))

# Replace the values in zi with new_data
abc.values = new_data

jd = abc.rename('jd')

## Merge Dataset

In [28]:
data_arrays = [
    water_relative_wind_speed.reset_coords(drop=True),
    zu.reset_coords(drop=True),
    sfc_temp_2m_ocean.reset_coords(drop=True),
    zt.reset_coords(drop=True),
    surface_pressure.reset_coords(drop=True),
    relative_humidity.reset_coords(drop=True),
    zq.reset_coords(drop=True),
    sea_water_temperature.reset_coords(drop=True),
    shortwave_down.reset_coords(drop=True),
    longwave_down.reset_coords(drop=True),
    latitude.reset_coords(drop=True),
    longitude.reset_coords(drop=True),
    zi.reset_coords(drop=True),
    rain_rate.reset_coords(drop=True),
    sea_surface_salinity.reset_coords(drop=True),
    jd.reset_coords(drop=True)
]

merged_dataset = xr.merge(data_arrays)

output_path = "/work/mh0731/m300868/10_COARE/ngc4008_to_COARE_trop_Pacific_2020_JAN_input.nc"
merged_dataset.to_netcdf(output_path)

merged_dataset

<xarray.Dataset> Size: 180MB
Dimensions:  (time: 30, cell: 74824)
Coordinates:
  * time     (time) datetime64[ns] 240B 2020-01-02 2020-01-03 ... 2020-01-31
  * cell     (cell) int64 599kB 0 1 2 3 4 5 ... 74819 74820 74821 74822 74823
Data variables: (12/16)
    u        (time, cell) float32 9MB 8.862 8.68 8.687 ... 5.08 4.907 4.977
    zu       (time, cell) float32 9MB 10.0 10.0 10.0 10.0 ... 10.0 10.0 10.0
    t        (time, cell) float32 9MB 26.34 26.32 26.31 ... 23.88 23.85 23.86
    zt       (time, cell) float32 9MB 2.0 2.0 2.0 2.0 2.0 ... 2.0 2.0 2.0 2.0
    P        (time, cell) float32 9MB 1.015e+03 1.015e+03 ... 1.01e+03 1.01e+03
    rh       (time, cell) float32 9MB 0.7062 0.7102 0.7079 ... 0.7516 0.7505
    ...       ...
    lat      (time, cell) float64 18MB 13.09 13.25 13.4 ... -0.2984 -0.1492
    lon      (time, cell) float64 18MB 150.1 150.3 150.1 ... 225.2 224.8 225.0
    zi       (time, cell) float64 18MB 600.0 600.0 600.0 ... 600.0 600.0 600.0
    rain     (time, cell) float32 9MB 0.03522 0.02669 ... 1.007e-09 1.866e-08
    Ss       (time, cell) float32 9MB 34.16 34.16 34.15 ... 35.09 35.08 35.09
    jd       (time, cell) int64 18MB 1 1 1 1 1 1 1 1 ... 30 30 30 30 30 30 30 30